In [ ]:
django-admin startproject incomeexpensesapi

In [ ]:
cd incomeexpensesapi

python manage.py runserver

stop server and install;
pip install djangorestframework

In [ ]:
we create an app that can help us manage our users- called authentication
python manage.py startapp authentication


in incomeexpensesapi settings.py we register this newly created app
in installed apps, add
"rest_framework",
"authentication", 

In [ ]:
we create our custom user model
in the authentication app
in models.py 

from django.contrib.auth.models import (AbstractBaseUser, BaseUserManager, PermissionsMixin)

02:56
now we set up a user manager

class UserManager(BaseUserManager):

    def create_user(self, username, email, password= None):

        if username is None:
            raise TypeError('Users must have a username')
        if email is None: 
            raise TypeError('Users must have an email')




In [ ]:
Now we define how a user shall be created
class UserManager(BaseUserManager):

    def create_user(self, username, email, password= None):

        if username is None:
            raise TypeError('Users must have a username')
        if email is None: 
            raise TypeError('Users must have an email')
        
        user = self.model(username= username, email= self.normalize_email(email))
        #now we set a password for the user
        user.set_password(password)
        user.save()
        return user

In [ ]:
now we create a superuser/ admin
    def create_superuser(self, username, email, password= None):

        if password is None:
            raise TypeError('password must be set')
        
        
        user = self.create_user(username, email, password)
        user.is_super_user = True
        user.is_staff = True
        user.save()

        return user

 

In [ ]:
now we setup our model class
#10:15
class User(AbstractBaseUser, PermissionsMixin):
    username= models.CharField(max_length= 300, unique = True, db_index= True)
#db_index= True makes us able to search for uses by username
    email= models.EmailField(max_length= 300, unique = True, db_index= True)
    is_verified= models.BooleanField( default= False)    
    #tells us that initially, before verification, all users are unverified
    is_active= models.BooleanField( default= True)    
    is_staff= models.BooleanField( default= False)  
    
    
##helps us know when the user profile was created  
    created_at= models.DateTimeField(auto_now_add= True)
    updated_at= models.DateTimeField(auto_now= True)
    
    

##now we define what attribute the user shall use to login
    USERNAME_FIELD = 'email'
    REQUIRED_FIELDS= ['username']
    
##tells us how to manage updates of the above login criteria
    objects = UserManager()
    
###to set up str method

def __str__(self):
    return self.email

####now we create a method called 'tokens' to be able to do something user the user's Token
    

def tokens (self):
    return ''    


In [ ]:
now in settings.py (for incomeexpensesapi) we must define the host user model

after 'ALLOWED_ HOSTS []'
AUTH_USER_MODEL = 'authentication.User'
####now we can  go to models.py(authentication app) and make use of our User class


In [ ]:
###TO REGISTER A USER
we shall be creating registration endpoints
in views.py of authentication app

from rest_framework import generics
#we create a registration class

class RegisterView(generics.GenericAPIView):
    
    #since we have first-time users, they shall be reistering new data, hence,we shall use a POST request
    def post(self, request):
        #now we get and use the data the new user has filled and sent
        user = request.data
       
        
    


In [ ]:
 #now we use a serializer class-- which we shall create in our authentication app and call serializers.py
 #in serializers.py
    from rest_framework import serializers
    from .models import User

    class RegisterSerializer(serializers.ModelSerializer):
        #now we define the fields in our serializer
        
        ##"write_only " because we dont want to send the password to the frontend
        

    class RegisterSerializer(serializers.ModelSerializer):
         password= serializers.CharField(max_length = 70, min_length= 8, write_only= True)
    
    #Meta defines our model
  
          class Meta:
              model= User
              fields= ['email', 'username', 'password']

     #now we can validate the user information
     def validate(self, attrrs):
          return super().validate(attrrs)

        

In [ ]:
in views.py
once we get the data,,,,,,,, we need to send it to the serializer



from django.shortcuts import render
from rest_framework import generics, status
from .serializers import RegisterSerializer
from rest_framework.response import Response


class RegisterView(generics.GenericAPIView): 
    serializer_class= RegisterSerializer



    def post(self, request):
        user = request.data
        serializer = self.serializer_class(data= user)
        serializer.is_valid(raise_exception= True)
        serializer.save()
        #05:18
after this save, we can store the saved data as #07:51
        user_data= serializer.data
    
        return Response (user_data, status= status.HTTP_201_CREATED)

In [ ]:
back in our serializers.py file

     def validate(self, attrrs):
        email= attrs.get('email', '')
        username= attrs.get('username', '')
        
        if not username.isalnum():
            raise serializers.ValidationError('Username must contain only alphanumeric')
        return attrs
        
    def create(self, validated_data):
        
#create gets called when we say serializer.save()
        return User.objects.create_user(**validated_data)

In [ ]:
after creating our endpoints/ views, we must register it in a new file iside our authentication app-- urls.py


from django.urls import path
from .views import RegisterView



urlpatterns = [
    path("register/", RegisterView.as_view(), name= "register"),
]

###now we hook this up with our main application routing #10:15

go to urls.py of the incomeexpensesapi



from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path("admin/", admin.site.urls),
    path('auth/', include('authentication.urls')),

]


#10:49


now we save;
         python manage.py makemigrations
         python manage.py migrate
         python manage.py runserver
         
        
open and run postman app
#12:30
http://127.0.0.1:8000/auth/register/
        POST request
        
13:11
         

In [ ]:
#14:41

setting up a new key
in settings.py of incomeexpensesapi

just above #password...

REST_FRAMEWORK= {
    'NON-FIELD_ERROR_KEY' = 'error'


}

In [ ]:
#in fourth video
import os
REST_FRAMEWORK= {
    'NON-FIELD_ERROR_KEY': 'error',
    'DEFAULT_AUTHENTICATION_CLASSES': (
        'rest_framework_simplejwt.authentication.JWTAuthentication',
    )
    
    }

in STATIC_URL...

EMAIL_USE_TLS = True
EMAIL_HOST = 'smtp.gmail.com'
EMAIL_PORT = 587
EMAIL_HOST_USER = os.environ.get('EMAIL_HOST_USER')
EMAIL_HOST_PASSWORD = os.environ.get('EMAIL_HOST_PASSWORD')
#04:45

In [ ]:
in views.py 
from rest_framework_simplejwt.tokens import RefreshToken
from .models import User
05:36
    
         
   
    
    


In [ ]:
in the authentication app, create a new file called utils.py
#06:41